In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

In [3]:
df_fake = pd.read_csv('Fake.csv')
df_true = pd.read_csv('True.csv')

In [4]:
df_fake.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [5]:
df_true.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [6]:
df_fake.nunique()

title      17903
text       17455
subject        6
date        1681
dtype: int64

In [7]:
df_true.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [8]:
df_true['class'] = 1
df_fake['class'] = 0

In [9]:
df_true.shape


(21417, 5)

In [10]:
for i in range(23480,23470-1):
    df_fake.drop([i],axis = 0,inplace = True)
for i in range(21416,21406-1):
    df_true.drop([i],axis = 0,inplace = True)

In [11]:
df_fake.shape,df_true.shape


((23481, 5), (21417, 5))

In [12]:
df_merge = pd.concat([df_fake,df_true],axis = 0)

In [13]:
df_merge.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [14]:
df_merge.tail()

,title,text,subject,date,class
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


In [15]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [16]:
df=df_merge.drop(['title','subject','date'],axis =1)

In [17]:
df.isnull().sum()

text     0
class    0
dtype: int64

In [18]:
df.reset_index(inplace = True)

In [19]:
df.drop(['index'],axis= 1,inplace = True)


In [20]:
df.columns

Index(['text', 'class'], dtype='object')

In [21]:
import string
import re
import nltk
from nltk.util import pr
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from nltk.corpus import stopwords
stemmer = nltk.SnowballStemmer('english') #stemmer helps in finding the rootword of a word and helpful for tagging
nltk.download('stopwords')
stopword = set(stopwords.words('english'))

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [22]:
def clean(text):
    text=str(text).lower()
    text=re.sub('\[.*?\}]','',text)
    text=re.sub('https?://\S+|www\.\S+','',text)
    text=re.sub('<.*?>+','',text)
    text=re.sub('[%s]'%re.escape(string.punctuation),'',text)
    text=re.sub('\n','',text)
    text=re.sub('\w*\d\w*','',text)
    text=[word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text=[stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text

In [23]:
df['text'] = df['text'].apply(clean)

In [24]:
x = df['text']
y = df['class']

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer #frequency of a word in the whole document ( Term Frequency-Inverse Document Frequency)
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x)# converting scalar string into vector
vectorizer.get_feature_names_out()
print(X.shape)

(44898, 177147)


In [26]:
first_vector = X[0]
dataframe = pd.DataFrame(first_vector.T.todense(),index =vectorizer.get_feature_names_out(),columns = ['tfidf'])
dataframe.sort_values(by = ['tfidf'],ascending=False)

,tfidf
decemb,0.444953
hater,0.332062
year,0.262646
enemi,0.200595
happi,0.196774
...,...
gazetauz,0.000000
gazett,0.000000
gazetteher,0.000000
gazettemail,0.000000


In [27]:
from sklearn.model_selection import train_test_split


In [28]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.25)

In [29]:
model = DecisionTreeClassifier()
model.fit(X_train,y_train)

DecisionTreeClassifier()

In [30]:
y_pred = model.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [31]:
print('training accuracy:',model.score(X_train,y_train))
print('testing accuracy:',model.score(X_test,y_test))

training accuracy: 0.9999703026163395
testing accuracy: 0.9976837416481069


In [32]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5906
           1       1.00      1.00      1.00      5319

    accuracy                           1.00     11225
   macro avg       1.00      1.00      1.00     11225
weighted avg       1.00      1.00      1.00     11225



In [33]:
from sklearn.ensemble import RandomForestClassifier

In [34]:
clf = RandomForestClassifier(n_estimators = 10)
clf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=10)

In [35]:
y_pred = clf.predict(X_test)
y_pred

array([0, 0, 1, ..., 0, 1, 0], dtype=int64)

In [36]:
print('training accuracy:',clf.score(X_train,y_train))
print('testing accuracy:',clf.score(X_test,y_test))


training accuracy: 0.99940605232679
testing accuracy: 0.9575946547884187


In [37]:
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
CLF = GradientBoostingClassifier(n_estimators= 100,random_state = 42)
CLF.fit(X_train,y_train)
y_pred = CLF.predict(X_test)
y_pred

In [ ]:
print('training accuracy:',CLF.score(X_train,y_train))
print('testing accuracy:',CLF.score(X_test,y_test))


In [ ]:
from sklearn.linear_model import LogisticRegression 

In [ ]:
clff = LogisticRegression()

In [ ]:
clff.fit(X_train,y_train)

In [ ]:
y_pred = clff.predict(X_test)


In [ ]:
print('testing accuracy:',clff.score(X_test,y_test))
print('training accuracy:',clff.score(X_train,y_train))
